In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_fscore_support
from sklearn.preprocessing import label_binarize
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV

In [16]:
# 1. 데이터 불러오기
data = pd.read_csv('azdigar nav sartirovka+0 delete.csv')

In [17]:
# 분석할 열들
columns = [
    'w08chronic_a', 'w08chronic_b', 'w08chronic_c',
    'w08chronic_d', 'w08chronic_e', 'w08chronic_f',
    'w08chronic_g', 'w08chronic_h', 'w08chronic_i',
    'w08chronic_k', 'w08chronic_l', 'w08chronic_m'
]

In [18]:
# 2. 데이터 클리닝 - 'w08chronic_m'에서 값이 3인 행 제거
cleaned_data = data[data['w08chronic_m'] != 3]

In [20]:
# 3. 피처 데이터 정의 (모든 chronic 컬럼 제외)
X = cleaned_data.drop(columns=columns)
y_all = cleaned_data[columns]

In [21]:
# 4. 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [22]:
# 5. 모델 정의
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'MLP': MLPClassifier(max_iter=2000, random_state=42)
}


In [23]:
# 6. 결과 저장을 위한 빈 데이터프레임 생성
result_table = pd.DataFrame(columns=['Target', 'Model', 'Precision', 'Recall', 'F1-Score', 'Accuracy', 'Macro F1', 'AUC'])

In [24]:
# 7. 각 타겟 열을 사용한 학습 및 평가
for target_column in columns:
    print(f'\n[{target_column}] 열을 예측:')
    print('==================================')

    y = cleaned_data[target_column].values

    # 데이터 나누기
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # 8. 데이터 불균형 처리 (RandomOverSampler 사용)
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

    # 9. PCA 또는 RFECV로 피처 선택 (PCA 사용)
    pca = PCA(n_components=10, random_state=42)
    X_train_pca = pca.fit_transform(X_train_resampled)
    X_test_pca = pca.transform(X_test)

    # 타겟 클래스가 두 개 이상이면 AUC 계산을 위해 타겟을 이진화 (One-vs-Rest 방식)
    y_test_bin = label_binarize(y_test, classes=np.unique(y))
    if y_test_bin.shape[1] == 1:  # 클래스가 하나인 경우 (로지스틱 회귀처럼 처리 안 되게)
        y_test_bin = np.concatenate([1 - y_test_bin, y_test_bin], axis=1)

    # 10. 각 모델에 대해 학습 및 평가
    for model_name, model in models.items():
        print(f'\n[{model_name}] 모델 평가:')
        print('--------------------------')

        try:
            # 모델 학습
            model.fit(X_train_pca, y_train_resampled)

            # 예측
            y_pred = model.predict(X_test_pca)
            
            # 확률 예측 값이 있는 경우
            if hasattr(model, "predict_proba"):
                y_pred_proba = model.predict_proba(X_test_pca)
            else:
                y_pred_proba = np.zeros((len(y_test), len(np.unique(y_test))))  # 확률 값을 지원하지 않는 모델에 대한 처리

            # 정확도 계산
            accuracy = accuracy_score(y_test, y_pred)

            # precision, recall, f1-score 계산 (weighted average)
            precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

            # Macro F1 계산
            macro_f1 = f1_score(y_test, y_pred, average='macro')

            # AUC 계산 (다중 클래스의 경우 One-vs-Rest 방식 사용)
            try:
                auc = roc_auc_score(y_test_bin, y_pred_proba, multi_class="ovr", average='weighted')
            except ValueError:
                auc = np.nan  # AUC 계산 불가 시 처리

            # 결과 추가
            new_row = pd.DataFrame({
                'Model': [model_name],
                'Target': [target_column],
                'Precision': [precision],
                'Recall': [recall],
                'F1-Score': [f1],
                'Accuracy': [accuracy],
                'Macro F1': [macro_f1],
                'AUC': [auc]
            })
            result_table = pd.concat([result_table, new_row], ignore_index=True)
        except Exception as e:
            print(f"{model_name} 모델에서 오류 발생: {e}")
            continue


[w08chronic_a] 열을 예측:

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_b] 열을 예측:

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_c] 열을 예측:

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_d] 열을 예측:

[Random Forest] 모델 평가:
--------------------------


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_e] 열을 예측:

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_f] 열을 예측:

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_g] 열을 예측:

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_h] 열을 예측:

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_i] 열을 예측:

[Random Forest]

c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_l] 열을 예측:

[Random Forest] 모델 평가:
--------------------------


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_m] 열을 예측:

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------


In [25]:
# 11. 결과 테이블 출력
print("\n전체 모델 평가 결과:")
print(result_table)


전체 모델 평가 결과:
          Target          Model  Precision    Recall  F1-Score  Accuracy  \
0   w08chronic_a  Random Forest   0.619737  0.618102  0.618241  0.618102   
1   w08chronic_a  Decision Tree   0.548024  0.547461  0.547647  0.547461   
2   w08chronic_a            SVM   0.649743  0.644592  0.643926  0.644592   
3   w08chronic_a            MLP   0.602239  0.601545  0.601727  0.601545   
4   w08chronic_b  Random Forest   0.666357  0.730684  0.686020  0.730684   
5   w08chronic_b  Decision Tree   0.671090  0.673289  0.672178  0.673289   
6   w08chronic_b            SVM   0.712110  0.605960  0.636258  0.605960   
7   w08chronic_b            MLP   0.660261  0.629139  0.642881  0.629139   
8   w08chronic_c  Random Forest   0.859161  0.920530  0.888787  0.920530   
9   w08chronic_c  Decision Tree   0.869334  0.856512  0.862789  0.856512   
10  w08chronic_c            SVM   0.883556  0.731788  0.791888  0.731788   
11  w08chronic_c            MLP   0.875350  0.822296  0.846508  0.822296  